In [1]:
import glob
import os
import json

In [20]:
data_dir = "data/input/位置決め/alpha1/140/Qchem-SF-TD-bhhlyp-631gd+140.gjf"

paths = [p for p in glob.glob(data_dir)]
# direction = "+"
direction = data_dir.split("/")[-1].split("_")[0][24]
# polymorph = "alpha1"
polymorph = data_dir.split("/")[3]

In [5]:
# Z-matrix形式のデータに対し、二面角の符号を反転させます。

# ダミーを参照しているdについては操作しない→d5, d6, d7はそのままにする

start_key = "\nd8"
end_key = "\nConstants"

for path in paths:
    with open(path,"r") as f:
        data=f.read()
        # 文字列置換
        start = data.find(start_key)
        end = data.find(end_key)
        target_list = data[start:end]
        initial_d_list = []
        for dn in target_list.split('\n'):
            d_angle = dn.split(' ')[-1]
            initial_d_list.append(d_angle)
        # 空の要素を削除
        initial_d_list = list(filter(None, initial_d_list))
        # initial_d_listを使ってtarget_listを検索し、マッチした部分を-1倍する
        for initial_d in initial_d_list:
            data = data.replace(initial_d, str(float(initial_d) * -1))
    # 同じファイル名で保存
    with open(path, mode="w", encoding="cp932") as f:
        f.write(data)

In [21]:
# directionは周辺分子を貼り付けるだけならどちらでもいい
# direction = "+"
# polymorph = "alpha2"

# 周辺分子を貼り付ける。多形ごとのデータを別ファイルで保管し、importして使う。
json_open = open("data/SA_data (" + direction + ").json", "r")
json_load = json.load(json_open)
polymorph_info = json_load[polymorph]

# 中心分子のデータの後に書き足す
insert_key = "d30\n"
start_key = "d30\n"

for path in paths:             
    with open(path,"r") as f:
        data=f.read()
        # 文字列置換
        data = data.replace(insert_key, start_key + polymorph_info["surrounding molecule"])
    with open(path, mode="w", encoding="cp932") as f:
        f.write(data)
        

# cell constantの書き換え
start_key = "\nConstants"
end_key = "-90.0000\n"

for path in paths:             
    with open(path,"r") as f:
        data=f.read()
        
        start = data.find(start_key)
        end = data.find(end_key) + len(end_key)
        target = data[start:end]
        # 文字列置換
        data = data.replace(target, "\n" + polymorph_info["cell constant"])
    with open(path, mode="w", encoding="cp932") as f:
        f.write(data)
        
# ダミーを参照するパラメーターの書き換え
for path in paths:             
    with open(path,"r") as f:
        data=f.read()
        # 文字列置換するパラメーター
        r5 = data[data.find("\nr5"):data.find("\nr6")]
        a5 = data[data.find("\na5"):data.find("\na6")]
        d5 = data[data.find("\nd5"):data.find("\nd6")]
        a6 = data[data.find("\na6"):data.find("\na7")]
        d6 = data[data.find("\nd6"):data.find("\nd7")]
        d7 = data[data.find("\nd7"):data.find("\nd8")]
#         置換
        p_list = [r5, a5, d5, a6, d6, d7]
        for p in p_list:
            data = data.replace(p, "\n" + p[1:6] + polymorph_info[p[1:3]])
    with open(path, mode="w", encoding="cp932") as f:
        f.write(data)
